In [ ]:
# # Create session  
# session = isf.InspireFaceSession(opt, isf.HF_DETECT_MODE_ALWAYS_DETECT)  
  
# # Detect faces  
# faces = session.face_detection(image)  
  
# # Extract bounding box coordinates  
# for face in faces:  
#     x1, y1, x2, y2 = face.location  # Bounding box coordinates  
#     center = ((x1 + x2) / 2, (y1 + y2) / 2)  
#     size = (x2 - x1, y2 - y1)  
#     angle = face.roll  # Face rotation angle
#     rect = ((center[0], center[1]), (size[0], size[1]), angle)
#     box = cv2.boxPoints(rect).astype(int)
#     cv2.drawContours(draw, [box], 0, (100, 180, 29), line_thickness)

# # Extract face embeddings  
# feature = session.face_feature_extract(image, face)

In [ ]:
import os
import numpy as np
import onnxruntime as ort
import cv2
import torch
import sys
sys.path.append("/home/yash-sisodia/face-detection/insightface/recognition/arcface_torch")
from backbones import get_model
from ground_truth import GROUND_TRUTH, GROUND_TRUTH_BRANDS, GROUND_TRUTH_CREATORS

COLLAB_DIR = "/home/yash-sisodia/face-detection/collaborators_india/collaborators"

ms1_sess = ort.InferenceSession("/home/yash-sisodia/face-detection/ms1mv2_r100.onnx")
glint_sess = ort.InferenceSession("/home/yash-sisodia/face-detection/glint360k_r100.onnx")

@torch.no_grad()
def extract_embeddings_onnx(image_path, sess):
    try:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (112, 112))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.transpose(img, (2, 0, 1)).astype(np.float32)
        img = img / 255.0
        img = (img - 0.5) / 0.5
        img = np.expand_dims(img, axis=0)
        input_name = sess.get_inputs()[0].name
        emb = sess.run(None, {input_name: img})[0]
        return emb.squeeze()
    except Exception:
        print(f"Couldn't load {image_path}")
        return []

usernames = [d for d in os.listdir(COLLAB_DIR) if os.path.isdir(os.path.join(COLLAB_DIR, d))]

for idx, USERNAME in enumerate([u for u in usernames if u in GROUND_TRUTH], start=1):
    PROFILE_IMAGE = os.path.join(COLLAB_DIR, USERNAME, "profile.jpg")
    THUMBNAIL_DIRS = [
        os.path.join(COLLAB_DIR, USERNAME, "latestPosts"),
        os.path.join(COLLAB_DIR, USERNAME, "latestIgtvVideos"),
    ]

    prof_ms1, prof_glint = [], []
    thumb_ms1, thumb_glint = [], []

    if os.path.exists(PROFILE_IMAGE):
        print(f"\n=== [{idx}] Extracting for {USERNAME}")
        prof_ms1 = extract_embeddings_onnx(PROFILE_IMAGE, ms1_sess)
        prof_glint = extract_embeddings_onnx(PROFILE_IMAGE, glint_sess)
    else:
        print(f"\n[{idx}] No profile image found for {USERNAME}")

    for folder_path in THUMBNAIL_DIRS:
        if os.path.exists(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith((".jpg")):
                    img_path = os.path.join(folder_path, file)
                    emb_ms1 = extract_embeddings_onnx(img_path, ms1_sess)
                    emb_glint = extract_embeddings_onnx(img_path, glint_sess)
                    if isinstance(emb_ms1, np.ndarray) and emb_ms1.size > 0:
                        thumb_ms1.append(emb_ms1)
                    if isinstance(emb_glint, np.ndarray) and emb_glint.size > 0:
                        thumb_glint.append(emb_glint)
        else:
            print(f"[{idx}] {USERNAME}: Missing {os.path.basename(folder_path)} folder")

    np.savez(
        os.path.join(COLLAB_DIR, USERNAME, "embeddings_ms1mv2.npz"),
        profile_embeddings=np.array([prof_ms1]) if len(prof_ms1) else np.empty((0,512)),
        thumbnail_embeddings=np.array(thumb_ms1) if thumb_ms1 else np.empty((0,512)),
    )
    np.savez(
        os.path.join(COLLAB_DIR, USERNAME, "embeddings_glint360k.npz"),
        profile_embeddings=np.array([prof_glint]) if len(prof_glint) else np.empty((0,512)),
        thumbnail_embeddings=np.array(thumb_glint) if thumb_glint else np.empty((0,512)),
    )


=== [1] Extracting for ellement.company

=== [2] Extracting for ajiolife

=== [3] Extracting for emprallofficial
[3] emprallofficial: Missing latestIgtvVideos folder

=== [4] Extracting for offduty.india

=== [5] Extracting for ___tripathi__

=== [6] Extracting for himalaya_facecare

=== [7] Extracting for lashupindia
[7] lashupindia: Missing latestIgtvVideos folder

=== [8] Extracting for beromtnails
[8] beromtnails: Missing latestIgtvVideos folder

=== [9] Extracting for fuaarkofficial

=== [10] Extracting for ohsogaga
[10] ohsogaga: Missing latestPosts folder
[10] ohsogaga: Missing latestIgtvVideos folder

=== [11] Extracting for staywrogn

=== [12] Extracting for ___nikhat____
[12] ___nikhat____: Missing latestIgtvVideos folder

=== [13] Extracting for _.dracarys.x_
[13] _.dracarys.x_: Missing latestIgtvVideos folder

=== [14] Extracting for spykarofficial

=== [15] Extracting for riseformen
[15] riseformen: Missing latestIgtvVideos folder

=== [16] Extracting for herbyangeloffici

In [ ]:
import os
import numpy as np
from PIL import Image
import insightface
import json

from insightface.utils import download  
download('models', 'antelopev2', root='~/.insightface') 

COLLAB_DIR = "/home/yash-sisodia/face-detection/collaborators_india/collaborators"

model = insightface.app.FaceAnalysis(
    name="antelopev2",
    providers=['CPUExecutionProvider']
)
model.prepare(ctx_id=0, det_size=(640, 640))


def extract_embeddings(image_path):
    try:
        img = np.array(Image.open(image_path).convert("RGB"))
    except Exception as e:
        print(f"Couldn't load")
        return []

    faces = model.get(img)
    embeddings = []
    for i, face in enumerate(faces):
        embeddings.append(face.normed_embedding)
    return embeddings

usernames = [d for d in os.listdir(COLLAB_DIR) if os.path.isdir(os.path.join(COLLAB_DIR, d))]

GROUND_TRUTH_BRANDS = {
    "mynykaa",
    "nykaanaturals",
    "nykaaskinrx",
    "nykaacosmetics",
    "wowskinscienceindia",
    "mamaearth.in",
    "letspurplle",
    "asabeautyindia",
    "trysugar",
    "nathabit.in",
    "discover.pilgrim",
    "vlccin",
    "forestessentials",
    "faebeautyofficial",
    "thebodyshopindia",
    "lovecolorbar",
    "biotique_world",
    "maccosmeticsindia",
    "revlon_india",
    "bobbibrownindia",
    "himalaya_facecare",
    "plumgoodness",
    "plumbodylovin",
    "disguisecosmetics",
    "myglamm",
    "coloressenceofficial",
    "rubys.organics",
    "elle18_india",
    "livon",
    "herbalessencesindia",
    "thebeautycoindia",
    "loverecode",
    "facescanada",
    "clinique_in",
    "suroskiebeauty",
    "paccosmetic",
    "litcosmeticsindia",
    "beauty_secret_highend_makeup",
    "biodermaindia",
    "mymakeupstory_india",
    "londonprimeindia",
    "charactercosmeticsindia",
    "forever52india",
    "keautybeauty.in",
    "shopaarel",
    "justgoldindia",
    "lashupindia",
    "saturnhealth.ghc",
    "skinkraftshop",
    "marshealth.ghc",
    "beromtnails",
    "blissible_cosmetics",
    "prishebeauty",
    "kindedbeauty",
    "abelia.cares",
    "farmbeautyofficial",
    "ellement.company",
    "skinpassword.in",
    "kaybykatrina",
    "82e.official",
    "arata.in",
    "herbyangelofficial",
    "nutriglowcosmetics",
    "oh_ind",
    "kapiva_official",
    "mcaffeineofficial",
    "perforaofficial",
    "sephora_india",
    "recodeeverydaymakeup",
    "officialswissbeauty",
    "mycosiq",
    "suganda.co",
    "myskinq",
    "auliglow",
    "theformularx",
    "himairag",
    "dermabayskincare",
    "beardo.official",
    "themancompany",
    "reginaldmen.grooming",
    "bombayshavingcompany",
    "mybombae__",
    "yaanman.official",
    "allman.in",
    "dollarshaveclub",
    "oldspice",
    "oldspiceindia",
    "oldspicehair",
    "oldspiceuk",
    "foxtaleskin",
    "ohsogaga",
    "riseformen",
    "dotandkey.skincare",
    "gush.beauty",
    "try.moody",
    "moxiebeautyofficial",
    "poshanskincare",
    "offduty.india",
    "emprallofficial",
    "littlebox.india",
    "tokyo_talkies",
    "freakinsindia",
    "newme.asia",
    "uptownie101",
    "crayy.heads",
    "bonkers.corner",
    "streetstylestoreofficial",
    "maincharacter_india",
    "manyavar",
    "izfworld",
    "virgio.official",
    "staywrogn",
    "theindiangarageco_",
    "ajiolife",
    "fabindiaofficial",
    "bibaindia",
    "allensollyindia",
    "wforwoman",
    "raymond_the_complete_man",
    "spykarofficial",
    "siyaramsindia",
    "kalkifashion",
    "bunaai",
    "libasindia",
    "shopmulmul",
    "houseofmasaba",
    "midsummer.india",
    "endlessummer.shop",
    "shopaamili",
    "layrrd",
    "kicaactive",
    "fuaarkofficial",
    "zymratwear",
    "ovrtrn",
    "birdeye.india",
    "hrxbrand",
    "Chapter2drip",
    "homedepot",
    "walmart",
    "target",
    "bestbuy",
    "loweshomeimprovement",
    "costco",
    "amazon",
    "kohls",
    "tjmaxx"
}
GROUND_TRUTH_CREATORS = {
    "_.dracarys.x_",
    "_.harshitha_.gowda._",
    "_.hett",
    "_.keshavi._",
    "_.kirranchauhan._",
    "_.life_in_pastel._",
    "_.manisha_kulal",
    "_.mile.y",
    "_.minney__06",
    "_.miss_komal_.1026",
    "_.nickyy75_",
    "_.nutrihealth",
    "_.palakvasudeva._",
    "_.phenomenail",
    "_.poojjja._",
    "_.priyanshiii01",
    "_.ranideepa._",
    "_.samridhii_",
    "_.shikhachaudhary._",
    "_.shobhomita",
    "_.shrush_",
    "_.srilakshmi.__",
    "_.sudeshnax.__",
    "_.tanisha._.22._",
    "_.vandana._03",
    "_3.4ooo",
    "__.aaddyyyyyyyy",
    "__.aishi_",
    "__.ambiverttt",
    "__.anuraaag",
    "__.mitara.b.__",
    "__.prachi.__26",
    "__.sannss",
    "__.shammu_zz",
    "__.soulzy",
    "__.tashaa.__",
    "___ .ishpreet",
    "____.jagriti.___",
    "______jasmine1_______",
    "_____keppy_____",
    "_____tequila_girl",
    "____sanjana___boro____",
    "____vartikaa___",
    "___aamu___05",
    "___antara__ghosh___",
    "___chinmoy_bora",
    "___dhanu07___",
    "___dubey___ji___",
    "___jasmine_05",
    "___jelly_bean___",
    "___kamakshiiiiii",
    "___malemnganbiii___",
    "___maxumilian___",
    "___mikku___",
    "___nikhat____",
    "___notyourcupoftea___",
    "___riyuhhh___",
    "___sadah____",
    "___simran__official___",
    "___srn_9779_",
    "___tannu20",
    "___tripathi__",
    "___twinkling___",
    "___whatarethose",
    "__akshitagupta",
    "__ankitaa._____",
    "__apurvapawar_",
    "__ar_yaaa__",
    "__barbie_grl",
    "__baruah__96",
    "__beauty__squad__",
    "__bellabeautyy",
    "__bhumi_thakur___",
    "__bint.abdullah__",
    "__brownie_love__",
    "__chachiii_",
    "__chikka",
    "__clematiss__",
    "__darshan____0024",
    "__devlina__",
    "__dr__.aastha___.yadav",
    "__embxx",
    "__estrellaaaa.__",
    "__hussainujjain",
    "__iamleeba._",
    "__ishannnnn._",
    "__ishikasachdeva__",
    "__ishsh__",
    "__jenny_pereira__",
    "__jess_mavalia__",
    "__jessferns__43",
    "__kajal.bisht__",
    "__kalitapriyamr__",
    "__kaur___simran",
    "__khushiiiii__3",
    "__kripali",
    "__lil.jannat",
    "__lucy.2.0__",
    "__ma.dh.u___",
}
GROUND_TRUTH = GROUND_TRUTH_BRANDS.union(GROUND_TRUTH_CREATORS)

for idx, USERNAME in enumerate([u for u in usernames if u in GROUND_TRUTH], start=1):
    PROFILE_IMAGE = os.path.join(COLLAB_DIR, USERNAME, "profile.jpg")
    THUMBNAIL_DIRS = [
        os.path.join(COLLAB_DIR, USERNAME, "latestPosts"),
        os.path.join(COLLAB_DIR, USERNAME, "latestIgtvVideos"),
    ]
    OUTPUT_FILE = os.path.join(COLLAB_DIR, USERNAME, "embeddings_output.npz")

    profile_embeddings = []
    if os.path.exists(PROFILE_IMAGE):
        print(f"\nExtracting from {USERNAME}")
        profile_embeddings = extract_embeddings(PROFILE_IMAGE)
    else:
        print("\nNo profile image found")

    thumbnail_embeddings = []
    for folder_path in THUMBNAIL_DIRS:
        if os.path.exists(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith((".jpg", ".jpeg", ".png")):
                    image_path = os.path.join(folder_path, file)
                    thumbnail_embeddings.extend(extract_embeddings(image_path))
        else:
            print(f"\nNo thumbnails folder found at {folder_path}")

    np.savez(
        OUTPUT_FILE,
        profile_embeddings=np.array(profile_embeddings),
        thumbnail_embeddings=np.array(thumbnail_embeddings),
    )

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/yash-sisodia/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/yash-sisodia/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/yash-sisodia/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/yash-sisodia/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/yash-sisodia/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None